In [64]:
import json
import numpy as np
import pandas as pd
import os
import glob
import shutil
from tqdm import tqdm
#import japanize_matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'
#from sciplotlib import style as spstyle
import seaborn as sns
from sklearn.model_selection import train_test_split,StratifiedGroupKFold

def search_object(sentence,organ):
    ##構造化文章単位を入力として、特定臓器の情報が入っているかを調べる。
    ##入力 sentence
    ##出力  IO+AEのtoken,IO token,AE token,IOのcertainty_score,IOのtype
    img_obs = sentence['clinical_object']
    mdfs = sentence['modifiers']
    ae_tokens = []

    for i in range(len(mdfs)):
        mdf = mdfs[i]
        if mdf['type'] == 'Anatomical_entity':
            ae_tokens += mdf['tokens']
    
    keywords = ae_tokens+img_obs['tokens']
    #print(keywords)
    if organ in keywords:
        keywords = ''.join(ae_tokens+img_obs['tokens'])
        io_tokens = ''.join(img_obs['tokens'])
        ae_tokens = ''.join(ae_tokens)
        return keywords,io_tokens,ae_tokens,img_obs['certainty_score'],img_obs['type']
    else:
        return None,None,None,None,None
def search_clinicalfinding(modifiers):
    ##clinical_descriptor clinical_discriptorが文中に含まれているかを判別
    ##入力 modifiers
    ##出力  含まれるかどうかbool,clinicalと判断されたwords(なければ0を入れる。)
    for i in range(len(modifiers)):
        modifier = modifiers[i]
        if modifier['type'] == 'Clinical_finding':
            return modifier['certainty_score'],''.join(modifier['tokens'])
        else:continue
    return None,'not_found'
def to_multi_label(df,abnormal_list):
    for abnormal_label in abnormal_list:
        df[abnormal_label] = df['label'].apply(lambda x: abnormal_label in x).astype(int)
    df['nofinding'] = df[abnormal_list].sum(axis=1) == 0
    return df

In [98]:
###全体のラベル作成
###observationは絶対に文中にあるという前提。
###

df = pd.read_pickle('../server_data/all_data_20211001-20220605.zip').drop_duplicates(subset='jmid_name').reset_index(drop=True)

organs = ['心','肝','胆','膵','脾','副腎','腎']
os.makedirs('../output',exist_ok=True)
counter = 0
jmid_names = df['jmid_name']
json_lists = df['FINDING_JSON']
#def detect_change(modifiers):
change_word_list = []
clinical_word_list = []
data = []
counter = 0
for jmid_name,json_load in zip(jmid_names,json_lists):
    for organ in organs:
        #print(organ)
        for i in range(len(json_load)):
            all_keyword,io_keyword,ae_keyword,certainty,img_type = search_object(json_load[i],organ)
            clinical_scale,clinical_word = search_clinicalfinding(json_load[i]['modifiers'])

            
            if all_keyword is not None:
                counter+=1
                data.append([jmid_name,organ,all_keyword,io_keyword,ae_keyword,certainty,img_type,clinical_word,clinical_scale])
            else:
                continue

data = pd.DataFrame(data,columns=['jmid_name','organ','all_tokens','io_tokens','ae_tokens','obs_certainty','obs_name','clin_findings','clin_certainty'])
data['jmid_name'] = data['jmid_name'].apply(lambda x:x.split('/')[-1])

In [99]:
data_kidney = data[(data['obs_name'] == 'Imaging_observation') & (data['organ'] == '腎')].reset_index(drop=True)

In [100]:
print([i for i in data_kidney[data_kidney['obs_certainty'] == 0]['io_tokens'].value_counts().index])

['異常', '水腎', '所見', 'リンパ節腫大', '異常所見', '腫瘤', '腎腫瘤', '病変', '有意な異常', '腫大リンパ節', '結石', '腹水', '粗大病変', 'mass_lesion', '腎腫大', '事項', '狭窄', '造影不領域', '腫瘤性病変', '造影効果', '腫大', '水腎水尿管', '腎結石', '腫瘤形成', '腫瘤影', '血栓形成', '腎腫瘍', '腫瘍栓', '壁肥厚', '液貯留', '骨破壊像', '前立腺腫大', '萎縮', '腎嚢胞', 'filling_defect', '造影不良', '嚢胞', '造影不良域', 'SOL', '副腎腫瘤', '輪郭不整', '腎動脈瘤', '脂肪成分', '骨硬化性病変', '脂肪濃度', '肥厚', '像', 'beak_sign', '閉塞機転', '副腎腫大', '尿排泄', '動脈瘤', '血腫', '尿管拡張', '尿管結石', '副腎腫瘍', '腎排泄', '拡張', '有意なリンパ節', '脂肪織濃度上昇', '造影低下', '腎損傷', '実質腫瘤', '早期濃染', '腎内腫瘤', '腎腫瘤病変', '腎尿管結石', '腎増強不良域', '粗大腫瘤', 'リンパ節腫脹', '腎萎縮', '結節', '脂肪織の混濁', '副路発達水腎', 'リンパ節', '腎静脈腫瘍栓', '異常壁肥厚像', '腎充実性腫瘤', '解離', '事項（-）', '毛羽立ち', '腎洞病変', '菲薄化', '大半', '腹水貯留', '瘤影', '腹水の貯留', '出血性嚢胞', '充実腫瘤', '腫瘍', '拡張像', '濃度上昇', '充実性腫瘤', '濃度上昇+結節', '高濃度域', '胸腹水', '腎機能低下', '異常ガス像', 'LDA', '遅延', '構造', '腎盂拡張', '腸管壁肥厚', '尿路結石', '異常ない異常', 'free_air', '水腎なし傍大動脈リンパ節腫大', '血栓', '結節病変', '異常病変', '異常像', '肥大', '有意なサイズの腫脹', '造影欠損像', '線不透過性結石', '有意な萎縮', '変形', '脂肪濃度上昇の進展', '有意なサイズ', '腹水増加水腎', '腎動静脈瘻', 'た

In [101]:
print([i for i in data_kidney[data_kidney['obs_certainty'] != 0]['io_tokens'].value_counts().index])

['腎嚢胞', '結石', '嚢胞', '小嚢胞', '萎縮', '腎萎縮', '腎結石', '腫瘤', '変形', '石灰化', '拡張', '腫大', '結節', '小結石', '水腎', 'シャント', '水腎症', '腎のう胞', '造影効果', '造影不良域', '早期濃染', '軟部影', '微小結石', 'complicated_cyst', '腎盂嚢胞', '小結節', '毛羽立ち', '腎腫大', '脂肪織混濁', 'LDA', '脂肪織濃度上昇', '子宮筋腫', '病変', '微小嚢胞', '動脈瘤', 'リンパ節', '馬蹄腎', '解離', '造影不領域', '凍結域', '像', 'SOL', '腹水', '腎動脈瘤', '狭窄', '移植腎', '腎瘢痕', '瘤状拡張', '壁肥厚', '小リンパ節', '水腎水尿管', '腎腫瘤', '筋腫', '菲薄化', '発達', 'washout', '脂肪成分', '液貯留', '液体貯留', '脂肪', '結腸憩室', '結節影', '高吸収域', '低吸収域', '腫瘤影', '高吸収', '傍腎盂嚢胞', '濃染', '脂肪腫', '血腫', '嚢胞腎', '腹部大動脈瘤', '嚢胞性病変', '脾腎シャント', 'シャント発達', '腫瘤性病変', '脂肪濃度', '血管筋脂肪腫', '壁在血栓', '腎シャント', '大腸憩室', '狭小化', 'AML', '肥厚', '傍腎盂のう胞', '腫大リンパ節', '前立腺腫大', 'air', '出血性嚢胞', '脾腫', '萎縮瘢痕像', 'wash_out', '出血性', 'リンパ節腫大', '圧排', '小石灰化', '毛羽立ち像', '瘤', '血栓', '増強不良域', '低濃度域', '腎下部大動脈瘤', '腎病変', '腎高度萎縮', '腎部分萎縮', '脂肪濃度の結節', '軟部腫瘤', '腎complicated_cyst', '微小石灰化', '造影効果低下', '構造', '造影パターン', '胆石', '造影不良', '脂肪肝', '造影効果低下域', '混濁', '複雑嚢胞', '腎盂拡張', 'リピオドール残存', '腎下部腹部大動脈瘤', '瘢痕影', '嚢胞性腫瘤', '腎回転異常', '腹膜／腹壁

In [102]:
normal_obs_finding_list = ['異常', '水腎', '所見', '異常所見', '腫瘤', '病変', '腎腫瘤', '有意な異常', '結石', '粗大病変', '事項', 'mass_lesion', '腎腫大', '狭窄', '水腎水尿管', '造影不領域', '腫瘤性病変', '腫大', '腎結石', '腫瘤影', '腫瘤形成', '腎腫瘍', '壁肥厚', '造影不良域', '造影不良', '拡張', 'SOL', '輪郭不整', '萎縮', '嚢胞', '腎嚢胞', '像', '脂肪成分', '脂肪濃度', '肥厚', '脂肪織濃度上昇', '腎排泄',  '血腫', '腎内腫瘤', '結節', '腎尿管結石', '造影低下', '腎腫瘤病変', '腎萎縮', '腎損傷', '腎増強不良域', '脂肪織の混濁', '実質腫瘤', '粗大腫瘤', '事項（-）', '有意な萎縮', '腎充実性腫瘤', '菲薄化', '濃度上昇', '出血性嚢胞', '毛羽立ち', '腫瘍', '瘤影', '充実性腫瘤', '拡張像', '充実腫瘤', '腎洞病変', '濃度上昇+結節', '異常ない異常', '水腎症', '腎機能低下', '異常像', 'mass', '変形', '腎盂', 'LDA', '石灰化', '結節病変', '腎病変', 'ただちに異常', '異常病変', '占拠性病変', '腎盂拡張', '出血', '脂肪織濃度の上昇', '線不透過性結石', '高濃度域', '造影欠損像', '器質的病変', '腎腫瘤形成', '腎SOL']

In [103]:
abnormal_obs_finding_list =['結石', '腎嚢胞', '嚢胞', '小嚢胞', '萎縮', '腫瘤', '変形', '石灰化', '拡張', '腫大', '結節',
                        '腎結石', '造影効果', '造影不良域', '早期濃染', '腎のう胞', '軟部影',  'complicated_cyst', 'サンゴ状石灰化', 
                        '腎盂嚢胞', 'LDA', '毛羽立ち', '腎萎縮', '微小嚢胞', '凍結域','傍腎盂のう胞','毛羽立ち', '脂肪織混濁','脂肪織濃度上昇' ]
##疾患のグループにまとめる。

abnormal_obs_finding_id = [0,1,1,1,2,3,2,0,3,2,3,0,3,3,3,1,3,1,0,1,3,3,2,1,3,1,3,3,3,]
##グループのまとまりの名前をつける。まとまりごとに文字の包含関係がないようにする。→嚢胞と肝嚢胞というグループを作らない。

abnormal_obs_finding_list_renamed = ['結石', '嚢胞', '形態変化','other_abnormality']


**腎臓は左右があるので処理が特別**

In [104]:
##まず、右左(両)の指定があればそれだけ取り出す。
##腎のみのtokenでも、certainty=0ならば両方負例として取り出す。
##todo:現在「腎」tokenは構造化不良により負例しか取り出していない。正例も取り出せるようにしたい。
print(data_kidney.shape)
normal_and_nonspecified_anatomy = (data_kidney['obs_certainty'] == 0)#data_kidney['ae_tokens'].apply(lambda x:x == '') & (data_kidney['obs_certainty'] == 0)
print('nonspecific',normal_and_nonspecified_anatomy.sum())
data_kidney['left'] = (data_kidney['all_tokens'].str.contains('左腎'))|(data_kidney['all_tokens'].str.contains('両腎'))|normal_and_nonspecified_anatomy
data_kidney['right'] = (data_kidney['all_tokens'].str.contains('右腎'))|(data_kidney['all_tokens'].str.contains('両腎'))|normal_and_nonspecified_anatomy
print(data_kidney['right'].sum(),data_kidney['left'].sum(),(data_kidney['left']*data_kidney['right']).sum())


(38915, 9)
nonspecific 7886
25728 26884 19146


In [105]:
data_kidney.head(2)

,jmid_name,organ,all_tokens,io_tokens,ae_tokens,obs_certainty,obs_name,clin_findings,clin_certainty,left,right
0,jmid_0059925_0000.nii.gz,腎,両腎嚢胞,腎嚢胞,両,4,Imaging_observation,not_found,NaN,True,True
1,jmid_0016814_0000.nii.gz,腎,膵脾腎両腎嚢胞,腎嚢胞,膵脾腎両,4,Imaging_observation,not_found,NaN,True,True


In [106]:
## unpivot
data_kidney = pd.melt(data_kidney,id_vars=data_kidney.columns.values[:9],var_name='side',value_name='side_value').reset_index(drop=True)
data_kidney = data_kidney[data_kidney['side_value'] == True].reset_index(drop=True)
print(data_kidney.shape)

for i in tqdm(range(len(data_kidney))):
    data_kidney.loc[i,'jmid_name'] = data_kidney.loc[i,'jmid_name'].split('.')[0] + data_kidney.loc[i,'side'] +'.nii.gz'
print(data_kidney.shape)

available_segmentation_list = glob.glob('../data/kidney_seg_img/*.nii.gz') 
available_segmentation_list = [os.path.basename(p) for p in available_segmentation_list]
use_segmentation = data_kidney['jmid_name'].apply(lambda x:x in available_segmentation_list)
print(data_kidney.shape)

use_normalfindings = (data_kidney['io_tokens'].apply(lambda x:x in normal_obs_finding_list)) & (data_kidney['obs_certainty'] == 0)
use_abnormalfindings = (data_kidney['io_tokens'].apply(lambda x:x in abnormal_obs_finding_list)) & (data_kidney['obs_certainty'] != 0)

#use_abnormalfindings = (data_kidney['clin_findings'].str.contains('癌') | data_kidney['clin_findings'].str.contains('IPMN'))
use_certainty = ((data_kidney['obs_certainty'] == 0) | (data_kidney['obs_certainty'] == 4) )
data_kidney['use_training'] = (use_normalfindings|use_abnormalfindings)& use_segmentation 

data_kidney['label'] = 'not used'
data_kidney.loc[use_abnormalfindings,'label'] = data_kidney.loc[use_abnormalfindings,'io_tokens'].apply(lambda x:abnormal_obs_finding_list_renamed[abnormal_obs_finding_id[abnormal_obs_finding_list.index(x)]])
data_kidney.loc[use_normalfindings,'label'] = 'nofinding'

tmp = data_kidney.groupby('jmid_name')['label'].apply(lambda x: x.tolist())
data_kidney.drop('label',axis=1,inplace=True)
data_kidney = data_kidney.merge(tmp,how='left',on='jmid_name')



(52612, 11)


100%|██████████| 52612/52612 [00:02<00:00, 22185.26it/s]


(52612, 11)
(52612, 11)


In [107]:
data_kidney = to_multi_label(data_kidney,abnormal_obs_finding_list_renamed)

In [108]:
data_kidney.head(6)

,jmid_name,organ,all_tokens,io_tokens,ae_tokens,obs_certainty,obs_name,clin_findings,clin_certainty,side,side_value,use_training,label,結石,嚢胞,形態変化,other_abnormality,nofinding
0,jmid_0059925_0000left.nii.gz,腎,両腎嚢胞,腎嚢胞,両,4,Imaging_observation,not_found,NaN,left,True,True,[嚢胞],0,1,0,0,False
1,jmid_0016814_0000left.nii.gz,腎,膵脾腎両腎嚢胞,腎嚢胞,膵脾腎両,4,Imaging_observation,not_found,NaN,left,True,True,[嚢胞],0,1,0,0,False
2,jmid_0039412_0000left.nii.gz,腎,左腎嚢胞,腎嚢胞,左,4,Imaging_observation,not_found,NaN,left,True,True,[嚢胞],0,1,0,0,False
3,jmid_0038419_0000left.nii.gz,腎,両腎萎縮,腎萎縮,両,4,Imaging_observation,not_found,NaN,left,True,True,[形態変化],0,0,1,0,False
4,jmid_0004026_0000left.nii.gz,腎,左腎萎縮,萎縮,左腎,4,Imaging_observation,not_found,NaN,left,True,True,[形態変化],0,0,1,0,False
5,jmid_0032726_0000left.nii.gz,腎,左腎嚢胞,腎嚢胞,左,4,Imaging_observation,not_found,NaN,left,True,True,[嚢胞],0,1,0,0,False


In [109]:

#data_kidney = data_kidney[((data_kidney['obs_certainty'] == 0) & (data_kidney['abnormal'] == 0))|((data_kidney['obs_certainty'] == 4) & (data_kidney['abnormal'] == 1))]
#data_kidney = data_kidney[(data_kidney['side_value'])|(data_kidney['obs_certainty'] == 0)]
print(data_kidney.shape)
data_kidney = data_kidney.drop_duplicates(subset='jmid_name')
print('after removing duplicates',data_kidney.shape)


data_kidney_use = data_kidney[data_kidney['use_training'] == True].reset_index(drop=True)
data_kidney_use['file'] = data_kidney_use['jmid_name']
data_kidney_use['jmid_name'] = data_kidney_use['jmid_name'].apply(lambda x:'_'.join(x.split('_')[:-1])+'_0000.nii.gz')

print(data_kidney_use.shape)
data_kidney_use = data_kidney_use.merge(df,on='jmid_name',how='left')
print(data_kidney_use.shape)
data_kidney_use[abnormal_obs_finding_list_renamed+['nofinding']].sum(axis=0)

(52612, 18)
after removing duplicates (43998, 18)
(38481, 19)
(38481, 30)


結石                    4668
嚢胞                   23753
形態変化                  1853
other_abnormality      999
nofinding            10321
dtype: int64

In [ ]:
## cleanなラベルは検証データとして使用する。学習データはそれ以外を使用。
clean_df = pd.read_csv('../data/kidney_dataset_test_clean.csv')
jmid_files_clean = clean_df['file'].values
tmp = data_kidney_use[data_kidney_use['jmid_name'].apply(lambda x:x in jmid_files_clean)]
test_groups = tmp['FACILITY_CODE'].astype(str)+tmp['ACCESSION_NUMBER'].astype(str)
groups = data_kidney_use['FACILITY_CODE'].astype(str)+data_kidney_use['ACCESSION_NUMBER'].astype(str)


df_train = data_kidney_use[groups.apply(lambda x:x not in test_groups.tolist())]
df_test =  data_kidney_use[groups.apply(lambda x:x in test_groups.tolist())]


columns = ['file','organ','all_tokens','io_tokens','ae_tokens','obs_certainty','FINDING','FINDING_JSON','FACILITY_CODE','ACCESSION_NUMBER']+abnormal_obs_finding_list_renamed+['nofinding']
df_train[columns].sort_values(by='file').reset_index(drop=True).to_csv('../data/kidney_dataset_train_multi5.csv',index=False)
df_test[columns].sort_values(by='file').reset_index(drop=True).to_csv('../data/kidney_dataset_test_multi5.csv',index=False)